In [0]:
# Imports
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime, timedelta
import random

In [0]:
# ============================================
# Parameters
# ============================================
dbutils.widgets.text("total_campaigns", "228", "Total number of campaigns (18 ACTIVE + 210 FINISHED)")
dbutils.widgets.text("schema_name", "silver", "Silver schema name")
dbutils.widgets.text("campaigns_table", "marketing_campaigns", "Marketing campaigns table name")
dbutils.widgets.text("partition_date", "partition_date", "Partition Date (YYYY-MM-DD) - Business date for partitioning")
dbutils.widgets.text("processing_time", datetime.now().strftime("%Y-%m-%d") + " 00:00:00", "Processing time for campaign date logic (YYYY-MM-DD HH:MM:SS)")

total_campaigns = int(dbutils.widgets.get("total_campaigns"))
schema_name = dbutils.widgets.get("schema_name")
campaigns_table = dbutils.widgets.get("campaigns_table")
partition_date = dbutils.widgets.get("partition_date")
processing_time = datetime.strptime(dbutils.widgets.get("processing_time"), "%Y-%m-%d %H:%M:%S")

# Generate ETL date and time (when the job actually runs)
etl_datetime = datetime.now()
etl_date = etl_datetime.strftime("%Y-%m-%d")
etl_time = etl_datetime.strftime("%H:%M:%S")

print(f"Total Campaigns: {total_campaigns}")
print(f"Target: {schema_name}.{campaigns_table}")
print(f"Partition Date (Business Date): {partition_date}")
print(f"Processing Time: {processing_time}")
print(f"ETL Date (Job Run Date): {etl_date}")
print(f"ETL Time: {etl_time}")

In [0]:
# ============================================
# Configuration
# ============================================

# Merchant category ranges
merchant_category_ranges = {
    "Groceries & Supermarkets": (10000000, 10017999),
    "Restaurants & Fast Food": (10018000, 10033999),
    "Gas & Fuel": (10034000, 10043999),
    "Travel (Airline, Hotel, Car Rental)": (10044000, 10051999),
    "Healthcare & Pharmacies": (10052000, 10058899),
    "Retail – Clothing & Department Stores": (10058900, 10070999),
    "Online Marketplaces (Amazon, Etsy, etc.)": (10071000, 10083499),
    "Entertainment (Movies, Events, Digital Media)": (10083500, 10088499),
    "Utilities & Bills (Electric, Telecom, Internet)": (10088500, 10094499),
    "Other (Government, Insurance, Misc.)": (10094500, 10099999)
}

# Campaign types
campaign_types = [
    "cashback", "bonus_points", "discount", "flash_sale", 
    "holiday_offer", "first_time_user_offer", "0_apr_financing", "membership_reward"
]

# Regions with distribution
regions = ["Northeast", "Southeast", "Midwest", "Southwest", "West"]
region_distribution = [0.20, 0.25, 0.15, 0.15, 0.25]

# Campaign status distribution
active_campaigns = 18
finished_campaigns = 210

In [0]:
# ============================================
# UDF Definitions
# ============================================

def generate_target_type(seed):
    """Generate target type: 50% CATEGORY, 50% REGION"""
    random.seed(seed)
    return "CATEGORY" if random.random() < 0.50 else "REGION"

def select_merchant_category(seed):
    """Randomly select a merchant category"""
    random.seed(seed)
    return random.choice(list(merchant_category_ranges.keys()))

def select_merchant_id(category, seed):
    """Select a random merchant ID within the category range"""
    random.seed(seed)
    start_id, end_id = merchant_category_ranges[category]
    return random.randint(start_id, end_id)

def select_campaign_type(seed):
    """Randomly select a campaign type"""
    random.seed(seed)
    return random.choice(campaign_types)

def select_target_region(seed):
    """Select target region based on distribution"""
    random.seed(seed)
    return random.choices(regions, weights=region_distribution, k=1)[0]

def generate_start_date(status, processing_time_str, seed):
    """Generate start date based on campaign status"""
    random.seed(seed)
    processing_time = datetime.strptime(processing_time_str, "%Y-%m-%d %H:%M:%S")
    
    if status == "ACTIVE":
        # Max 7 days before processing time, 80% less than 2 days
        if random.random() < 0.80:
            days_back = random.randint(0, 2)
        else:
            days_back = random.randint(3, 7)
        return (processing_time - timedelta(days=days_back)).strftime("%Y-%m-%d")
    else:  # FINISHED
        # Between 1 year older and processing_time - 7 days
        max_days_back = 365
        min_days_back = 7
        days_back = random.randint(min_days_back, max_days_back)
        return (processing_time - timedelta(days=days_back)).strftime("%Y-%m-%d")

def generate_end_date(status, start_date_str, processing_time_str, seed):
    """Generate end date based on campaign status and start date"""
    random.seed(seed)
    processing_time = datetime.strptime(processing_time_str, "%Y-%m-%d %H:%M:%S")
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    
    if status == "ACTIVE":
        # Max 7 days after processing time, 80% less than 2 days
        if random.random() < 0.80:
            days_forward = random.randint(0, 2)
        else:
            days_forward = random.randint(3, 7)
        end_date = processing_time + timedelta(days=days_forward)
    else:  # FINISHED
        # Between start_date and processing_time - 7 days
        max_end_date = processing_time - timedelta(days=7)
        
        # Ensure end_date is after start_date
        min_days_forward = 1
        max_days_forward = (max_end_date - start_date).days
        
        if max_days_forward < min_days_forward:
            max_days_forward = min_days_forward
            
        days_forward = random.randint(min_days_forward, max_days_forward)
        end_date = start_date + timedelta(days=days_forward)
    
    return end_date.strftime("%Y-%m-%d")

def generate_budget(seed):
    """Generate campaign budget between 100K and 1M"""
    random.seed(seed)
    return random.randint(100000, 1000000)

def generate_target_uplift(seed):
    """Generate target uplift between 5% and 10%"""
    random.seed(seed)
    return round(random.uniform(0.05, 0.10), 2)


# Register UDFs
target_type_udf = F.udf(generate_target_type, StringType())
merchant_category_udf = F.udf(select_merchant_category, StringType())
merchant_id_udf = F.udf(select_merchant_id, IntegerType())
campaign_type_udf = F.udf(select_campaign_type, StringType())
target_region_udf = F.udf(select_target_region, StringType())
start_date_udf = F.udf(generate_start_date, StringType())
end_date_udf = F.udf(generate_end_date, StringType())
budget_udf = F.udf(generate_budget, IntegerType())
target_uplift_udf = F.udf(generate_target_uplift, DoubleType())

In [0]:
# ============================================
# Generate Campaigns DataFrame
# ============================================

print("\n📊 Generating campaigns...")

# Create base DataFrame with campaign IDs
df_campaigns = spark.range(0, total_campaigns).withColumn("campaign_id", F.col("id")).drop("id")

# Add campaign status (18 ACTIVE, rest FINISHED)
df_campaigns = df_campaigns.withColumn(
    "campaign_status",
    F.when(F.col("campaign_id") < active_campaigns, "ACTIVE").otherwise("FINISHED")
)

# Add campaign trigger type (100% Dynamic/Auto-Triggered)
df_campaigns = df_campaigns.withColumn("campaign_trigger_type", F.lit("Dynamic/Auto-Triggered Campaigns"))

# Generate seeds for randomization
df_campaigns = df_campaigns.withColumn("seed_base", F.col("campaign_id") * 1000)

# Generate target type
df_campaigns = df_campaigns.withColumn("target_type", target_type_udf(F.col("seed_base")))

# Generate merchant category
df_campaigns = df_campaigns.withColumn(
    "merchant_category", 
    merchant_category_udf(F.col("seed_base") + 1)
)

# Generate merchant_id (NULL for CATEGORY or REGION, populated for other types if any)
df_campaigns = df_campaigns.withColumn(
    "merchant_id",
    F.when((F.col("target_type") == "CATEGORY") | (F.col("target_type") == "REGION"), F.lit(None).cast(IntegerType()))
    .otherwise(merchant_id_udf(F.col("merchant_category"), F.col("seed_base") + 2))
)

# Generate campaign type
df_campaigns = df_campaigns.withColumn(
    "campaign_type",
    campaign_type_udf(F.col("seed_base") + 3)
)

# Generate target region (only for REGION target type, NULL otherwise)
df_campaigns = df_campaigns.withColumn(
    "target_region",
    F.when(F.col("target_type") == "REGION", target_region_udf(F.col("seed_base") + 4))
    .otherwise(F.lit(None).cast(StringType()))
)

# Generate start_date
processing_time_str = processing_time.strftime("%Y-%m-%d %H:%M:%S")
df_campaigns = df_campaigns.withColumn(
    "start_date",
    start_date_udf(F.col("campaign_status"), F.lit(processing_time_str), F.col("seed_base") + 5)
)

# Generate end_date
df_campaigns = df_campaigns.withColumn(
    "end_date",
    end_date_udf(F.col("campaign_status"), F.col("start_date"), F.lit(processing_time_str), F.col("seed_base") + 6)
)

# Generate budget
df_campaigns = df_campaigns.withColumn(
    "campaign_budget",
    budget_udf(F.col("seed_base") + 7)
)

# Generate target uplift
df_campaigns = df_campaigns.withColumn(
    "target_uplift",
    target_uplift_udf(F.col("seed_base") + 8)
)

In [0]:
# ============================================
# Enforce Uniqueness for ACTIVE Campaigns
# ============================================

print("\n🔍 Enforcing uniqueness constraint for ACTIVE campaigns...")

# Add row number partitioned by (merchant_category, target_type, campaign_type) for ACTIVE campaigns only
window_spec = Window.partitionBy("merchant_category", "target_type", "campaign_type", "campaign_status").orderBy("campaign_id")

df_campaigns = df_campaigns.withColumn(
    "row_num",
    F.when(F.col("campaign_status") == "ACTIVE", F.row_number().over(window_spec))
    .otherwise(F.lit(1))
)

# Keep only the first occurrence for ACTIVE campaigns (row_num = 1)
# For FINISHED campaigns, row_num is always 1, so they all pass through
df_campaigns_unique = df_campaigns.filter(F.col("row_num") == 1).drop("row_num")

# Count how many duplicates were removed
initial_count = df_campaigns.filter(F.col("campaign_status") == "ACTIVE").count()
final_count = df_campaigns_unique.filter(F.col("campaign_status") == "ACTIVE").count()
duplicates_removed = initial_count - final_count

print(f"   Initial ACTIVE campaigns: {initial_count}")
print(f"   Final ACTIVE campaigns: {final_count}")
print(f"   Duplicates removed: {duplicates_removed}")

# Replace df_campaigns with the deduplicated version
df_campaigns = df_campaigns_unique





# Add partition_date (business date for partitioning)
df_campaigns = df_campaigns.withColumn("partition_date", F.lit(partition_date).cast(DateType()))

# Add ETL metadata columns (when the job actually runs)
df_campaigns = df_campaigns.withColumn("etl_date", F.lit(etl_date).cast(DateType()))
df_campaigns = df_campaigns.withColumn("etl_time", F.lit(etl_time).cast(StringType()))

# Drop seed column
df_campaigns = df_campaigns.drop("seed_base")

# Reorder columns
df_campaigns = df_campaigns.select(
    "campaign_id",
    "target_type",
    "merchant_category",
    "merchant_id",
    "campaign_type",
    "campaign_trigger_type",
    "campaign_status",
    "start_date",
    "end_date",
    "campaign_budget",
    "target_uplift",
    "target_region",
    "partition_date",  # Business date - Partition column
    "etl_date",        # ETL run date
    "etl_time"         # ETL run time
)

In [0]:
# ============================================
# Write to Delta Table (Managed, Partitioned)
# ============================================

print(f"\n💾 Writing to Delta managed table: {schema_name}.{campaigns_table}")

(
    df_campaigns
    .filter(F.col("partition_date") == partition_date)       # keep only that partition's rows
    .write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"partition_date = '{partition_date}'")
    .saveAsTable(f"{schema_name}.{campaigns_table}")
)

print(f"✅ Successfully overwritten partition {partition_date} in {schema_name}.{campaigns_table}")